In [2]:
from gurobipy import Model, GRB, quicksum

def biweekly_fleet_size_planning(input_data):
    availability_costs = input_data["availability_costs"]
    move_costs_empty = input_data["move_costs_empty"]
    move_costs_loaded = input_data["move_costs_loaded"]
    penalties = input_data["penalties"]
    demand_scenarios = input_data["demand_scenarios"]
    cap1 = input_data["first_week_capacity_limits"]
    cap2 = input_data.get("second_week_capacity_limits", cap1)
    demand_week1 = input_data.get("first_week_demand", None)

    n = len(availability_costs)
    L = range(n)
    S = range(len(demand_scenarios))

    model = Model("BiWeeklyFleetPlanning")
    model.setParam("OutputFlag", 0)

    z = model.addVars(L, lb=0.0, vtype=GRB.INTEGER)
    x1_empty = model.addVars(L, L, lb=0.0, vtype=GRB.INTEGER)
    x1_load = model.addVars(L, L, lb=0.0, vtype=GRB.INTEGER)
    end1 = model.addVars(L, lb=0.0, vtype=GRB.INTEGER)
    x2_empty = model.addVars(L, L, S, lb=0.0, vtype=GRB.INTEGER)
    x2_load = model.addVars(L, L, S, lb=0.0, vtype=GRB.INTEGER)
    end2 = model.addVars(L, S, lb=0.0, vtype=GRB.INTEGER)
    w_plus = model.addVars(L, S, lb=0.0)
    w_minus = model.addVars(L, S, lb=0.0)

    model.addConstrs(
        x1_empty[i, j] + x1_load[i, j] <= cap1[i][j]
        for i in L for j in L
    )

    if demand_week1 is not None:
        model.addConstrs(
            x1_load[i, j] >= demand_week1[i][j]
            for i in L for j in L
        )

    model.addConstrs(
        quicksum(x1_empty[i, j] + x1_load[i, j] for j in L) <= z[i]
        for i in L
    )

    model.addConstrs(
        end1[i] == z[i]
        + quicksum(x1_empty[j, i] + x1_load[j, i] for j in L)
        - quicksum(x1_empty[i, j] + x1_load[i, j] for j in L)
        for i in L
    )

    model.addConstrs(
        x2_empty[i, j, s] + x2_load[i, j, s] <= cap2[i][j]
        for s in S for i in L for j in L
    )

    model.addConstrs(
        x2_load[i, j, s] >= demand_scenarios[s][1][i][j]
        for s in S for i in L for j in L
    )

    model.addConstrs(
        quicksum(x2_empty[i, j, s] + x2_load[i, j, s] for j in L) <= end1[i]
        for s in S for i in L
    )

    model.addConstrs(
        end2[i, s] == end1[i]
        + quicksum(x2_empty[j, i, s] + x2_load[j, i, s] for j in L)
        - quicksum(x2_empty[i, j, s] + x2_load[i, j, s] for j in L)
        for s in S for i in L
    )

    model.addConstrs(
        end2[i, s] - z[i] == w_plus[i, s] - w_minus[i, s]
        for s in S for i in L
    )

    model.setObjective(
        quicksum(availability_costs[i] * z[i] for i in L)
        + quicksum(
            move_costs_empty[i][j] * x1_empty[i, j] + move_costs_loaded[i][j] * x1_load[i, j]
            for i in L for j in L
        )
        + quicksum(
            demand_scenarios[s][0] * (
                quicksum(
                    move_costs_empty[i][j] * x2_empty[i, j, s] + move_costs_loaded[i][j] * x2_load[i, j, s]
                    for i in L for j in L
                )
                + quicksum(penalties[i] * (w_plus[i, s] + w_minus[i, s]) for i in L)
            )
            for s in S
        ),
        GRB.MINIMIZE
    )

    model.optimize()

    if model.Status == GRB.OPTIMAL:
        return model.ObjVal
    return None


In [5]:
import json

with open("sample.json", "r") as f:
    data = json.load(f)

if isinstance(data, list):
    if len(data) == 0:
        raise ValueError("sample.json is an empty list")
    data = data[0]

inp = data["input"]

input_data = {
    "availability_costs": inp["availability_costs"],
    "move_costs_empty": inp["move_costs_empty"],
    "move_costs_loaded": inp["move_costs_loaded"],
    "penalties": inp["penalties"],
    "first_week_capacity_limits": inp["first_week_capacity_limits"],
    "demand_scenarios": [
        (sc["probability"], sc["demand"])
        for sc in inp["demand_scenarios"]
    ]
}

if "second_week_capacity_limits" in inp:
    input_data["second_week_capacity_limits"] = inp["second_week_capacity_limits"]

if "first_week_demand" in inp:
    input_data["first_week_demand"] = inp["first_week_demand"]

obj = biweekly_fleet_size_planning(input_data)

print("Optimal expected total cost:", obj)
print("Reference output in JSON:", data.get("output"))


Optimal expected total cost: 268.22512905050456
Reference output in JSON: [268.22512905050456]


In [6]:
data

{'input': {'availability_costs': [11, 15, 8],
  'move_costs_empty': [[0, 2, 2], [1, 0, 1], [1, 2, 0]],
  'move_costs_loaded': [[0, 2, 4], [3, 0, 3], [4, 3, 0]],
  'penalties': [9, 7, 10],
  'first_week_capacity_limits': [[4, 6, 4], [5, 6, 6], [3, 3, 4]],
  'demand_scenarios': [{'probability': 0.798429434343465,
    'demand': [[1, 3, 2], [1, 2, 3], [3, 1, 1]]},
   {'probability': 0.2015705656565351,
    'demand': [[3, 3, 2], [3, 1, 1], [1, 3, 2]]}]},
 'output': [268.22512905050456]}

In [4]:
import random
import json

def generate_biweekly_instance(n, num_scenarios, filename, seed=0):
    random.seed(seed)

    availability_costs = [random.randint(5, 15) for _ in range(n)]
    penalties = [random.randint(4, 10) for _ in range(n)]

    move_costs_empty = [
        [0 if i == j else random.randint(1, 3) for j in range(n)]
        for i in range(n)
    ]

    move_costs_loaded = [
        [0 if i == j else random.randint(2, 5) for j in range(n)]
        for i in range(n)
    ]

    first_week_capacity_limits = [
        [random.randint(3, 6) for _ in range(n)]
        for _ in range(n)
    ]

    demand_scenarios = []
    probs = [random.random() for _ in range(num_scenarios)]
    total = sum(probs)
    probs = [p / total for p in probs]

    for s in range(num_scenarios):
        demand = [
            [random.randint(1, 3) for _ in range(n)]
            for _ in range(n)
        ]
        demand_scenarios.append({
            "probability": probs[s],
            "demand": demand
        })

    data = {
        "input": {
            "availability_costs": availability_costs,
            "move_costs_empty": move_costs_empty,
            "move_costs_loaded": move_costs_loaded,
            "penalties": penalties,
            "first_week_capacity_limits": first_week_capacity_limits,
            "demand_scenarios": demand_scenarios
        },
        "output": [268.22512905050456]
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=2)

generate_biweekly_instance(3, 2, "sample.json", seed=433445524)
